<a href="https://colab.research.google.com/github/aleehussnain/PythonProjects/blob/main/Bert_Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm.auto import tqdm
from tqdm.notebook import tqdm

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Tweets.csv']))

In [ ]:
df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
df.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [ ]:
 df.groupby('airline_sentiment').describe()

tweet_id                                            \
                     count          mean           std           min   
airline_sentiment                                                      
negative            9178.0  5.692602e+17  7.572474e+14  5.675900e+17   
neutral             3099.0  5.691841e+17  8.068405e+14  5.675883e+17   
positive            2363.0  5.691006e+17  8.112476e+14  5.676555e+17   

                                                                           \
                            25%           50%           75%           max   
airline_sentiment                                                           
negative           5.686511e+17  5.695355e+17  5.698798e+17  5.703106e+17   
neutral            5.684852e+17  5.693080e+17  5.699305e+17  5.703093e+17   
positive           5.684331e+17  5.691980e+17  5.698788e+17  5.703093e+17   

                  airline_sentiment_confidence            ...  \
                                         count      mean  ...   
airline_sentiment                                         ...   
negative                                9178.0  0.933365  ...   
neutral                                 3099.0  0.823303  ...   
positive                                2363.0  0.872039  ...   

                  negativereason_confidence      retweet_count            \
                                        75%  max         count      mean   
airline_sentiment                                                          
negative                                1.0  1.0        9178.0  0.093375   
neutral                                 0.0  0.0        3099.0  0.060987   
positive                                0.0  0.0        2363.0  0.069403   

                                                       
                        std  min  25%  50%  75%   max  
airline_sentiment                                      
negative           0.792865  0.0  0.0  0.0  0.0  44.0  
neutral            0.658037  0.0  0.0  0.0  0.0  28.0  
positive           0.659914  0.0  0.0  0.0  0.0  22.0  

[3 rows x 32 columns]

In [ ]:
df['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [ ]:
tweets=df[["airline_sentiment","text"]]
tweets

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,positive,@AmericanAir thank you we got on a different f...
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...
14637,neutral,@AmericanAir Please bring American Airlines to...
14638,negative,"@AmericanAir you have my money, you change my ..."


In [ ]:
postive_sentiment = tweets[tweets['airline_sentiment']=="positive"]

In [ ]:
postive_sentiment

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,positive,"@VirginAmerica it was amazing, and arrived an ..."
11,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...
...,...,...
14623,positive,@AmericanAir Love the new planes for the JFK-L...
14625,positive,@AmericanAir Flight 236 was great. Fantastic c...
14628,positive,Thank you. “@AmericanAir: @jlhalldc Customer R...
14630,positive,@AmericanAir Thanks! He is.


In [ ]:
negative_sentiment = tweets[tweets['airline_sentiment']=="negative"]
negative_sentiment

,airline_sentiment,text
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
15,negative,@VirginAmerica SFO-PDX schedule is still MIA.
17,negative,@VirginAmerica I flew from NYC to SFO last we...
...,...,...
14631,negative,@AmericanAir thx for nothing on getting us out...
14633,negative,@AmericanAir my flight was Cancelled Flightled...
14634,negative,@AmericanAir right on cue with the delays👌
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...


In [ ]:
nuetral_sentiment = tweets[tweets['airline_sentiment']=="neutral"]
nuetral_sentiment

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
2,neutral,@VirginAmerica I didn't today... Must mean I n...
7,neutral,@VirginAmerica Really missed a prime opportuni...
10,neutral,@VirginAmerica did you know that suicide is th...
23,neutral,@VirginAmerica will you be making BOS&gt;LAS n...
...,...,...
14607,neutral,@AmericanAir i need someone to help me out
14611,neutral,"@AmericanAir guarantee no retribution? If so, ..."
14632,neutral,"“@AmericanAir: @TilleyMonsta George, that does..."
14637,neutral,@AmericanAir Please bring American Airlines to...


In [ ]:
postive_sentiment

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,positive,"@VirginAmerica it was amazing, and arrived an ..."
11,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...
...,...,...
14623,positive,@AmericanAir Love the new planes for the JFK-L...
14625,positive,@AmericanAir Flight 236 was great. Fantastic c...
14628,positive,Thank you. “@AmericanAir: @jlhalldc Customer R...
14630,positive,@AmericanAir Thanks! He is.


In [ ]:
nuetral_sentiment = nuetral_sentiment.sample(postive_sentiment.shape[0])
nuetral_sentiment

,airline_sentiment,text
3766,neutral,@united when southwest did this to us -- a mec...
13068,neutral,@AmericanAir but my friend was told she was ab...
6537,neutral,@SouthwestAir can i take my electric skateboar...
7241,neutral,"@JetBlue no one yet but some, fl 8162"
681,neutral,@united booked award tix on ThaiAirways but I'...
...,...,...
7755,neutral,@JetBlue will we be able to watch the Oscars o...
5338,neutral,@SouthwestAir greetings
12393,neutral,@americanair One question.. my wife didn't get...
489,neutral,@virginamerica Any way to change from Main Cab...


In [ ]:
negative_sentiment = negative_sentiment.sample(nuetral_sentiment.shape[0])
negative_sentiment

,airline_sentiment,text
9397,negative,@usairways im trying to get my dads wheelchair...
1404,negative,"@united Not to belabor my point,but shouldn't ..."
12493,negative,@AmericanAir thanks for the canned reply.
4079,negative,@united was told by Supervisor that United dis...
3341,negative,"@united ""We like hearing from you."" So why hav..."
...,...,...
9510,negative,@USAirways I'm sure the people are working ver...
10799,negative,@nrhodes85: look! Another apology. DO NOT FLY ...
9288,negative,@USAirways I will never fly us airways again.
2243,negative,@united website says my flight is on time. It ...


In [ ]:
balanced_tweets = pd.concat([postive_sentiment, negative_sentiment,nuetral_sentiment])

In [ ]:
balanced_tweets.shape

(7089, 2)

In [ ]:
balanced_tweets

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,positive,"@VirginAmerica it was amazing, and arrived an ..."
11,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...
...,...,...
7755,neutral,@JetBlue will we be able to watch the Oscars o...
5338,neutral,@SouthwestAir greetings
12393,neutral,@americanair One question.. my wife didn't get...
489,neutral,@virginamerica Any way to change from Main Cab...


In [ ]:
balanced_tweets['airline_sentiment'].value_counts()

positive    2363
negative    2363
neutral     2363
Name: airline_sentiment, dtype: int64

In [ ]:
balanced_tweets['sentiments']=balanced_tweets['airline_sentiment'].apply(lambda x: 1 if x=='positive' else (0 if x=='negative' else 2))
balanced_tweets.sample(5)
#lambda x: x*10 if x<2 else (x**2 if x<4 else x+10)

,airline_sentiment,text,sentiments
11295,negative,@USAirways you used to have an evening flight ...,0
11024,negative,@USAirways ruined our honeymoon by causing us ...,0
7698,negative,@JetBlue can we get a honest update on departu...,0
13059,positive,@AmericanAir i got a new reservation for tomor...,1
8260,negative,@JetBlue can you give me an update on flight 1...,0


In [ ]:
balanced_tweets['sentiments']=balanced_tweets['airline_sentiment'].apply(lambda x: 1 if x=='positive' else (0 if x=='negative' else 2))
balanced_tweets.sample(5)


,airline_sentiment,text,sentiments
8698,neutral,@JetBlue to End Partnership with American Expr...,2
7933,neutral,@JetBlue can you share where it is coming from?,2
8808,neutral,@JetBlue Are departing flights from Boston hea...,2
1412,negative,@united bags left behind because plane overwe...,0
100,neutral,@VirginAmerica trying to add my boy Prince to ...,2


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 136.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased') 

In [ ]:

balanced_tweets['text'].iloc[0]

"@VirginAmerica plus you've added commercials to the experience... tacky."

In [ ]:
X_input_ids= np.zeros((len(balanced_tweets),256))
X_attn_masks = np.zeros((len(balanced_tweets),256))

In [ ]:

X_input_ids.shape

(7089, 256)

In [ ]:
#method to generate trainding data

In [ ]:
def generate_training_data(df, ids,masks, tokenizer):
  for i, text in tqdm(enumerate(balanced_tweets['text'])):
    tokenized_text = tokenizer.encode_plus(
        text,
        max_length=256,
        truncation= True,
        padding ='max_length',
        add_special_tokens = True,
        return_tensors='tf'
    )
    #print(tokenized_text)
    ids[i,:]=tokenized_text.input_ids 
    masks[i,:]= tokenized_text.attention_mask 
    #print(ids)
  return ids,masks
        
    

In [ ]:
X_input_ids, X_attn_masks = generate_training_data (balanced_tweets,X_input_ids, X_attn_masks, tokenizer)


0it [00:00, ?it/s]

In [ ]:
X_attn_masks

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [ ]:
X_input_ids

array([[  101.,   137.,  6567., ...,     0.,     0.,     0.],
       [  101.,   137.,  6567., ...,     0.,     0.,     0.],
       [  101.,   137., 16946., ...,     0.,     0.,     0.],
       ...,
       [  101.,   137.,  1821., ...,     0.,     0.,     0.],
       [  101.,   137., 16946., ...,     0.,     0.,     0.],
       [  101.,   137., 10280., ...,     0.,     0.,     0.]])

In [ ]:
labels =np.zeros((len(balanced_tweets),3))
#labels.shape

In [ ]:
balanced_tweets

,airline_sentiment,text,sentiments
1,positive,@VirginAmerica plus you've added commercials t...,1
6,positive,"@VirginAmerica yes, nearly every time I fly VX...",1
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D",1
9,positive,"@VirginAmerica it was amazing, and arrived an ...",1
11,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...,1
...,...,...,...
7755,neutral,@JetBlue will we be able to watch the Oscars o...,2
5338,neutral,@SouthwestAir greetings,2
12393,neutral,@americanair One question.. my wife didn't get...,2
489,neutral,@virginamerica Any way to change from Main Cab...,2


In [ ]:
labels[np.arange(len(balanced_tweets)),balanced_tweets['sentiments'].values]=1

In [ ]:
labels.shape

(7089, 3)

In [ ]:
#create dataset using tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])

In [ ]:
dataset= tf.data.Dataset.from_tensor_slices((X_input_ids,X_attn_masks,labels))

In [ ]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [ ]:
def SentimentDatasetMapFunction(input_ids, attn_masks,labels):
  return {
      'input_ids' : input_ids,
      'attention_mask' : attn_masks}, labels 
  

In [ ]:
dataset = dataset.map(SentimentDatasetMapFunction)

In [ ]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder = True)

In [ ]:
tr= 0.8
train_size =int((len(balanced_tweets)//16)*tr)

In [ ]:
train_size

354

In [ ]:
train_dataset=dataset.take(train_size)
val_dataset=dataset.skip(train_size)

In [ ]:
val_dataset

<_SkipDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 3), dtype=tf.float64, name=None))>

In [ ]:
from transformers import TFBertModel

In [ ]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attention_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')
bert_embedings = bert_model.bert(input_ids, attention_masks)[1]
intermediate_layer = tf.keras.layers.Dense(512, activation='relu',name='intermediate_layer')(bert_embedings)
output_layer = tf.keras.layers.Dense(3, activation='softmax',name ='output_layer')(intermediate_layer)
model = tf.keras.Model(inputs=[input_ids,attention_masks], outputs=output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
optim = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('Accuracy')

In [ ]:
model.compile(optimizer=optim, loss=loss_func,metrics=[acc])


In [ ]:
hist = model.fit( train_dataset,
                 validation_data = val_dataset,
                 epochs = 1
                 )

 44/354 [==>...........................] - ETA: 2:51:27 - loss: 3.9210 - Accuracy: 0.3111